In [1]:
!pip install pandas-ta
!pip install scikeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=88e43ec6358a9132c05eb90343ce603276d72702b4a36836ea66e5e1a871c775
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [2]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import plotly.graph_objects as go
from plotly.subplots import make_subplots

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
csv_files = ['/kaggle/input/global-stock-market-2008-present/2008_Globla_Markets_Data.csv', '/kaggle/input/global-stock-market-2008-present/2009_Globla_Markets_Data.csv', '/kaggle/input/global-stock-market-2008-present/2010_Global_Markets_Data.csv', '/kaggle/input/global-stock-market-2008-present/2011_Global_Markets_Data.csv', '/kaggle/input/global-stock-market-2008-present/2012_Global_Markets_Data.csv', '/kaggle/input/global-stock-market-2008-present/2013_Global_Markets_Data.csv', '/kaggle/input/global-stock-market-2008-present/2014_Global_Markets_Data.csv', '/kaggle/input/global-stock-market-2008-present/2015_Global_Markets_Data.csv',
             '/kaggle/input/global-stock-market-2008-present/2016_Global_Markets_Data.csv', '/kaggle/input/global-stock-market-2008-present/2017_Global_Markets_Data.csv', '/kaggle/input/global-stock-market-2008-present/2018_Global_Markets_Data.csv', '/kaggle/input/global-stock-market-2008-present/2019_Global_Markets_Data.csv', '/kaggle/input/global-stock-market-2008-present/2020_Global_Markets_Data.csv', '/kaggle/input/global-stock-market-2008-present/2021_Global_Markets_Data.csv', '/kaggle/input/global-stock-market-2008-present/2022_Global_Markets_Data.csv', '/kaggle/input/global-stock-market-2008-present/2023_Global_Markets_Data.csv']
data_frames = []

for file in csv_files:
    df = pd.read_csv(file)
    data_frames.append(df)

merged_df = pd.concat(data_frames, ignore_index=True)

merged_df.drop("Adj Close", axis=1, inplace=True)

merged_df["Date"] = pd.to_datetime(merged_df["Date"])
merged_df.sort_values(by="Date", inplace=True)

merged_df.to_csv("Total_Global_Markets_Data.csv", index=False)

In [4]:
df = merged_df[merged_df['Ticker'] == '^NSEI']
df.ta.rsi(close=df["Close"], length=14, append=True)
df.ta.macd(close=df["Close"], fast=12, slow=26, signal=9, append=True)
df

/tmp/ipykernel_24/56023630.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.ta.rsi(close=df["Close"], length=14, append=True)
/tmp/ipykernel_24/56023630.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.ta.macd(close=df["Close"], fast=12, slow=26, signal=9, append=True)
/tmp/ipykernel_24/56023630.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

,Ticker,Date,Open,High,Low,Close,Volume,RSI_14,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
3,^NSEI,2008-08-01,4331.600098,4422.950195,4235.700195,4413.549805,0.0,NaN,NaN,NaN,NaN
15,^NSEI,2008-08-04,4426.100098,4436.149902,4362.899902,4395.350098,0.0,NaN,NaN,NaN,NaN
27,^NSEI,2008-08-05,4395.799805,4515.149902,4376.000000,4502.850098,0.0,NaN,NaN,NaN,NaN
39,^NSEI,2008-08-06,4506.250000,4615.899902,4503.899902,4517.549805,0.0,NaN,NaN,NaN,NaN
51,^NSEI,2008-08-07,4515.250000,4580.149902,4493.700195,4523.850098,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
44843,^NSEI,2023-07-24,19748.449219,19782.750000,19658.300781,19672.349609,265600.0,63.755695,273.673029,7.441125,266.231903
44855,^NSEI,2023-07-25,19729.349609,19729.349609,19615.949219,19680.599609,369700.0,63.985942,259.151230,-5.664538,264.815769
44867,^NSEI,2023-07-26,19733.349609,19825.599609,19716.699219,19778.300781,244700.0,66.685075,252.614288,-9.761184,262.375473
44879,^NSEI,2023-07-27,19850.900391,19867.550781,19603.550781,19659.900391,334900.0,60.743643,235.168912,-21.765249,256.934160


In [5]:
years = df['Date'].dt.year.unique()
for year in years:
    year_data = df[df['Date'].dt.year == year]

    fig = go.Figure(data=[go.Candlestick(x=year_data['Date'], open=year_data['Open'],
                    high=year_data['High'], low=year_data['Low'], close=year_data['Close'])])
    fig.update_layout(
        title=f'Candlestick Chart - Year {year}', xaxis_title='Date', yaxis_title='Price')
    fig.show()

    fig = go.Figure(data=[go.Bar(x=year_data['Date'], y=year_data['Volume'],
                    text=year_data['Volume'], textposition='outside', marker_color='gold')])
    fig.update_layout(
        title=f'Volume - Year {year}', xaxis_title='Date', yaxis_title='Volume')
    fig.show()

    fig = go.Figure(data=[go.Scatter(x=year_data["Date"], y=year_data["RSI_14"],
                    mode="lines", name="RSI", line=dict(color="blue"))])
    fig.add_hline(y=70, line_dash="dash", line=dict(color='red'),
                  annotation_text='Overbought (70)', row=1, col=1)
    fig.add_hline(y=30, line_dash="dash", line=dict(color='green'),
                  annotation_text='Oversold (30)', row=1, col=1)
    fig.update_layout(
        title=f'RSI - Year {year}', xaxis_title='Date', yaxis_title='RSI')
    fig.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=year_data["Date"], y=year_data["MACD_12_26_9"],
                  mode="lines", name="MACD", line=dict(color="green")))
    fig.add_trace(go.Scatter(x=year_data["Date"], y=year_data["MACDs_12_26_9"],
                  mode="lines", name="Signal", line=dict(color="red")))
    fig.add_trace(go.Bar(x=year_data["Date"], y=year_data["MACDh_12_26_9"], name="Histogram", marker_color=np.where(
        year_data["MACDh_12_26_9"] < 0, "red", "green")))
    fig.update_layout(
        title=f'MACD - Year {year}', xaxis_title='Date', yaxis_title='RSI')
    fig.show()

In [6]:
nsei_data = merged_df[merged_df['Ticker']
                      == '^NSEI'][['Date', 'Ticker', 'Close']]
nsei_data.to_csv('NSEI_data.csv', index=False)

In [7]:
# Normalize the data
scaler = MinMaxScaler()

In [8]:
# Define the sequence length
seq_length = 5

# Define a function to create sequences and labels


def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length])
        labels.append(data[i + seq_length])
    return np.array(sequences), np.array(labels)

In [9]:
# Define a function to build the LSTM model
def build_lstm_model(units=50, dropout=0.1):
    model = Sequential()
    model.add(LSTM(units=units, return_sequences=True,
              input_shape=(seq_length, 1)))
    model.add(Dropout(dropout))
    model.add(LSTM(units=units))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [10]:
# Normalize the data
normalized_data_nsei = scaler.fit_transform(
    nsei_data['Close'].values.reshape(-1, 1))

# Split data into training and testing sets
train_size_nsei = int(len(normalized_data_nsei) * 0.8)
train_data_nsei, test_data_nsei = normalized_data_nsei[:
                                                       train_size_nsei], normalized_data_nsei[train_size_nsei:]

# Create sequences and labels
train_sequences_nsei, train_labels_nsei = create_sequences(
    train_data_nsei, seq_length)
test_sequences_nsei, test_labels_nsei = create_sequences(
    test_data_nsei, seq_length)

# Create a KerasRegressor wrapper for scikit-learn compatibility
model_nsei = KerasRegressor(model=build_lstm_model, units=50, dropout=0.1)

# Define the parameter grid
param_grid_nsei = {'batch_size': [16, 32], 'epochs': [
    50, 100], 'units': [50, 75], 'dropout': [0.1, 0.2]}

# Create the GridSearchCV object
grid_search_nsei = GridSearchCV(estimator=model_nsei, param_grid=param_grid_nsei, scoring=[
                                'r2', 'explained_variance', 'neg_mean_squared_error'], cv=3, refit='r2', verbose=1)

# Fit the search to training data
grid_search_nsei.fit(train_sequences_nsei, train_labels_nsei)

# Get the best parameters
best_params_nsei = grid_search_nsei.best_params_

# Train the final model with best parameters
best_model_nsei = build_lstm_model(
    units=best_params_nsei['units'], dropout=best_params_nsei['dropout'])
best_model_nsei.fit(train_sequences_nsei, train_labels_nsei,
                    epochs=best_params_nsei['epochs'], batch_size=best_params_nsei['batch_size'], verbose=0)

# Make predictions and plot for each feature
fig_nsei = make_subplots(rows=1, cols=1)

test_predictions_nsei = best_model_nsei.predict(test_sequences_nsei)
test_predictions_nsei = scaler.inverse_transform(test_predictions_nsei)
test_actual_nsei = scaler.inverse_transform(test_labels_nsei)

mse_nsei = mean_squared_error(test_actual_nsei, test_predictions_nsei)
mae_nsei = mean_absolute_error(test_actual_nsei, test_predictions_nsei)
rmse_nsei = np.sqrt(mse_nsei)
r2_nsei = r2_score(test_actual_nsei, test_predictions_nsei)
print(f"MSE: {mse_nsei:.2f}, MAE: {mae_nsei:.2f}, RMSE: {rmse_nsei:.2f}, R-squared: {r2_nsei:.2f}")

fig_nsei.add_trace(go.Scatter(x=nsei_data['Date'][train_size_nsei+seq_length:],
                   y=test_actual_nsei.ravel(), mode='lines', name=f'Actual'), row=1, col=1)
fig_nsei.add_trace(go.Scatter(x=nsei_data['Date'][train_size_nsei+seq_length:],
                   y=test_predictions_nsei.ravel(), mode='lines', name=f'Predicted'), row=1, col=1)
fig_nsei.update_layout(
    title='LSTM Time Series Forecasting for ^NSEI', height=800)
fig_nsei.show()

Fitting 3 folds for each of 16 candidates, totalling 48 fits
Epoch 1/50
123/123 [==============================] - 9s 6ms/step - loss: 0.0097
Epoch 2/50
123/123 [==============================] - 1s 6ms/step - loss: 2.3345e-04
Epoch 3/50
123/123 [==============================] - 1s 10ms/step - loss: 2.1926e-04
Epoch 4/50
123/123 [==============================] - 1s 8ms/step - loss: 2.1464e-04
Epoch 5/50
123/123 [==============================] - 1s 6ms/step - loss: 2.0760e-04
Epoch 6/50
123/123 [==============================] - 1s 6ms/step - loss: 1.9893e-04
Epoch 7/50
123/123 [==============================] - 1s 6ms/step - loss: 1.8766e-04
Epoch 8/50
123/123 [==============================] - 1s 6ms/step - loss: 1.8645e-04
Epoch 9/50
123/123 [==============================] - 1s 6ms/step - loss: 1.6461e-04
Epoch 10/50
123/123 [==============================] - 1s 6ms/step - loss: 1.5486e-04
Epoch 11/50
123/123 [==============================] - 1s 6ms/step - loss: 1.5918e-04
Epoch

In [11]:
# Create a date range from 31-07-2023 to 31-12-2024
start_date = '2023-07-31'
end_date = '2023-12-31'
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Filter out weekends (Saturday and Sunday)
business_days = date_range[date_range.weekday < 5]

# Create a DataFrame with the filtered dates
nsei_prediction_df = pd.DataFrame({'Date': business_days})

nsei_prediction_df['Ticker'] = '^NSEI'

In [12]:
# Normalize the data
scaler = MinMaxScaler()

In [13]:
nsei_final_data = pd.read_csv('NSEI_data.csv')

# Normalize the data
nsei_normalized_data = scaler.fit_transform(
    nsei_final_data['Close'].values.reshape(-1, 1))

# Create sequences and labels
nsei_train_sequences, nsei_train_labels = create_sequences(
    nsei_normalized_data, seq_length)

nsei_test_sequence = nsei_train_labels[-seq_length:]

model = best_model_nsei
params = best_params_nsei
model.fit(nsei_train_sequences, nsei_train_labels,
          epochs=params['epochs'], batch_size=params['batch_size'], verbose=0)

nsei_prediction_data = np.empty((0,))
for i in range(110):
    # Reshape the test sequence to match the input shape expected by the model
    nsei_test_sequence = nsei_test_sequence.reshape(1, -1, 1)
    nsei_prediction = model.predict(nsei_test_sequence)
    nsei_prediction_data = np.append(nsei_prediction_data, nsei_prediction)
    nsei_test_sequence = np.append(nsei_test_sequence, nsei_prediction)
    nsei_test_sequence = nsei_test_sequence[1:]

# Reshape and inverse transform the predictions
nsei_prediction_data = nsei_prediction_data.reshape(-1, 1)
nsei_prediction_data = scaler.inverse_transform(nsei_prediction_data)
nsei_prediction_data = nsei_prediction_data.ravel()

nsei_prediction_df['Close'] = nsei_prediction_data

nsei_prediction_df

1/1 [==============================] - 0s 23ms/step


,Date,Ticker,Close
0,2023-07-31,^NSEI,19636.192133
1,2023-08-01,^NSEI,19616.083356
2,2023-08-02,^NSEI,19596.900533
3,2023-08-03,^NSEI,19578.493845
4,2023-08-04,^NSEI,19562.760974
...,...,...,...
105,2023-12-25,^NSEI,19349.481851
106,2023-12-26,^NSEI,19349.467285
107,2023-12-27,^NSEI,19349.452719
108,2023-12-28,^NSEI,19349.442316


In [14]:
nsei_prediction_df.to_csv('NSEI_prediction.csv', index=False)

In [15]:
nsei_total_df = pd.concat([pd.read_csv('NSEI_data.csv'), pd.read_csv(
    'NSEI_prediction.csv')], ignore_index=True)
nsei_total_df["Date"] = pd.to_datetime(nsei_total_df["Date"])
nsei_total_df.sort_values(by="Date", inplace=True)

nsei_total_df = nsei_total_df[nsei_total_df['Date'].dt.year == 2023]

split_date = pd.to_datetime("28-07-2023")
nsei_before_split = nsei_total_df[nsei_total_df['Date'] <= split_date]
nsei_after_split = nsei_total_df[nsei_total_df['Date'] > split_date]

/tmp/ipykernel_24/1759251961.py:8: UserWarning:

Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



In [16]:
fig_final = go.Figure()
fig_final.add_trace(go.Scatter(
    x=nsei_before_split["Date"], y=nsei_before_split["Close"], mode="lines", name="Provided", line=dict(color="blue")))
fig_final.add_trace(go.Scatter(
    x=nsei_after_split["Date"], y=nsei_after_split["Close"], mode="lines", name="Predicted", line=dict(color="red")))
fig_final.update_layout(title='LSTM Closing Price Prediction ^NSEI',
                  xaxis_title='Date', yaxis_title='Price')
fig_final.show()